In [ ]:
import pandas as pd
import numpy as np
import requests
import json
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union
import geopandas as gpd
import osmnx as ox

In [2]:
df = pd.read_csv('datasets/rainfall_data_transformed_final.csv', encoding='utf-8')

In [3]:
df_long = df.melt(id_vars=['latitude','longitude'], var_name='datetime', value_name='rainfall')
df_long['datetime'] = pd.to_datetime(df_long['datetime'])

In [4]:
df_long['rainfall']=df_long['rainfall']*1000  # Convert from meters to mm

In [5]:
df_long['rainfall'].max()

60.70137

In [6]:
df_long['id'] = df_long.index

In [7]:
def filter_hp_uk_data(df):
    """
    Filter dataset to include only regions within Himachal Pradesh and Uttarakhand
    
    Parameters:
    df (pandas.DataFrame): Input dataframe with 'latitude' and 'longitude' columns
    
    Returns:
    pandas.DataFrame: Filtered dataframe
    """
    
    # Approximate bounding boxes for the states
    # Himachal Pradesh bounds
    hp_lat_min, hp_lat_max = 30.22, 33.25
    hp_lon_min, hp_lon_max = 75.47, 79.04
    
    # Uttarakhand bounds  
    uk_lat_min, uk_lat_max = 28.43, 31.45
    uk_lon_min, uk_lon_max = 77.34, 81.03
    
    # Create boolean masks for each state
    hp_mask = ((df['latitude'] >= hp_lat_min) & (df['latitude'] <= hp_lat_max) & 
               (df['longitude'] >= hp_lon_min) & (df['longitude'] <= hp_lon_max))
    
    uk_mask = ((df['latitude'] >= uk_lat_min) & (df['latitude'] <= uk_lat_max) & 
               (df['longitude'] >= uk_lon_min) & (df['longitude'] <= uk_lon_max))
    
    # Combine masks using OR operation
    combined_mask = hp_mask | uk_mask
    
    # Filter the dataframe
    filtered_df = df[combined_mask].copy()
    
    # Add a state column for identification
    filtered_df['state'] = 'Unknown'
    filtered_df.loc[hp_mask, 'state'] = 'Himachal Pradesh'
    filtered_df.loc[uk_mask, 'state'] = 'Uttarakhand'
    
    return filtered_df

In [ ]:
def download_osm_boundaries():
    """
    Download state boundaries from OpenStreetMap using OSMnx.
    """
    try:
        # Get boundaries for Himachal Pradesh and Uttarakhand
        # We can pass a list of place names to geocode_to_gdf
        places = ['Himachal Pradesh, India', 'Uttarakhand, India']
        gdf = ox.geocode_to_gdf(places)
        print("Downloaded OSM boundaries")
        
        # Filter for the specific states
        hp_boundary = gdf[gdf['name'] == 'Himachal Pradesh'].geometry
        uk_boundary = gdf[gdf['name'] == 'Uttarakhand'].geometry
        
        if not hp_boundary.empty and not uk_boundary.empty:
            return hp_boundary.iloc[0], uk_boundary.iloc[0]
        else:
            print("States not found in OpenStreetMap data")
            return None, None
            
    except Exception as e:
        print(f"Error downloading OSM data: {e}")
        return None, None

In [ ]:

def precise_boundary_filter(df, method='polygon'):
    """
    Main function to filter data using precise boundaries
    
    Parameters:
    df (pandas.DataFrame): Input dataframe
    method (str): 'polygon', 'osm', or 'natural_earth'
    
    Returns:
    pandas.DataFrame: Precisely filtered dataframe
    """
    
    print(f"Using {method} method for boundary filtering...")
    print(f"Original data points: {len(df)}")
    
    if method == 'polygon':
        # Use predefined polygons
        filtered_df = filter_hp_uk_data(df)
        
    elif method == 'osm':
        # Use OpenStreetMap boundaries
        hp_boundary, uk_boundary = download_osm_boundaries()
        print("Extracted OSM Boundaries")
        if hp_boundary and uk_boundary:
    # Create a GeoDataFrame from your DataFrame
            gdf = gpd.GeoDataFrame(
                df, geometry=gpd.points_from_xy(df['longitude'], df['latitude'])
            )
            gdf = gdf.set_crs('EPSG:4326')
            # Convert the boundaries to GeoDataFrames for the join
            hp_gdf = gpd.GeoDataFrame(geometry=[hp_boundary], crs='EPSG:4326')
            uk_gdf = gpd.GeoDataFrame(geometry=[uk_boundary], crs='EPSG:4326')

            # Perform geospatial joins to find points within the boundaries
            hp_points = gpd.sjoin(gdf, hp_gdf, how="inner", predicate="within")
            uk_points = gpd.sjoin(gdf, uk_gdf, how="inner", predicate="within")

            # Combine the results and add the state column
            filtered_df = pd.concat([hp_points, uk_points]).drop_duplicates(subset=['id']).copy()
            filtered_df['state'] = filtered_df.index.map(lambda idx: 'Himachal Pradesh' if idx in hp_points.index else 'Uttarakhand')

            # Optional: If you need a pandas DataFrame, convert it back
            filtered_df = pd.DataFrame(filtered_df.drop(columns=['geometry']))

        else:
            print("Failed to download OSM boundaries, falling back to polygon method")
            filtered_df = filter_hp_uk_data(df)

    print(f"Filtered data points: {len(filtered_df)}")
    print(f"Data retained: {len(filtered_df)/len(df)*100:.2f}%")

    if 'state' in filtered_df.columns:
        print("\nState-wise distribution:")
        print(filtered_df['state'].value_counts())
        
    return filtered_df

In [10]:
df_long=precise_boundary_filter(df_long, method='osm')

Using osm method for boundary filtering...
Original data points: 19193040
Downloaded OSM boundaries
Extracted OSM Boundaries
Filtered data points: 7290720
Data retained: 37.99%

State-wise distribution:
state
Himachal Pradesh    3733200
Uttarakhand         3557520
Name: count, dtype: int64


In [ ]:
import folium
import pandas as pd

def plot_sampled_rainfall_map(df, lat_col="latitude", lon_col="longitude", 
                              value_col="rainfall", time_col="datetime", 
                              state_col="state", sample_size=200):
    """
    Create an interactive Folium map with rainfall points,
    sampling 'sample_size' locations from each state (Uttarakhand, Himachal Pradesh).
    
    Parameters:
        df (pd.DataFrame): dataframe containing lat, lon, rainfall, datetime, state
        lat_col (str): latitude column
        lon_col (str): longitude column
        value_col (str): rainfall values column
        time_col (str): datetime column
        state_col (str): state column
        sample_size (int): number of points to sample per state
        
    Returns:
        folium.Map: interactive map object
    """
    # Filter states
    hp_df = df[df[state_col] == "Himachal Pradesh"].sample(n=min(sample_size, df[df[state_col]=="Himachal Pradesh"].shape[0]), random_state=42)
    uk_df = df[df[state_col] == "Uttarakhand"].sample(n=min(sample_size, df[df[state_col]=="Uttarakhand"].shape[0]), random_state=42)
    
    sampled_df = pd.concat([hp_df, uk_df])
    
    # Create base map
    m = folium.Map(location=[sampled_df[lat_col].mean(), sampled_df[lon_col].mean()], zoom_start=7)
    
    # Normalize rainfall for marker scaling
    max_val = sampled_df[value_col].max()
    if max_val == 0:
        max_val = 1
    
    for _, row in sampled_df.iterrows():
        popup_text = (
            f"<b>State:</b> {row[state_col]}<br>"
            f"<b>DateTime:</b> {row[time_col]}<br>"
            f"<b>Rainfall:</b> {row[value_col]:.4f}"
        )
        folium.CircleMarker(
            location=[row[lat_col], row[lon_col]],
            radius=4 + 10*(row[value_col]/max_val),  # scale by rainfall
            color="red" if row[state_col]=="Uttarakhand" else "blue",
            fill=True,
            fill_color="red" if row[state_col]=="Uttarakhand" else "blue",
            fill_opacity=0.6,
            popup=popup_text
        ).add_to(m)
    
    m.save("html_plots/sampled_rainfall_map.html")
    return m



In [ ]:
plot_sampled_rainfall_map(df_long)

In [13]:
df_long.sample(10) 

,latitude,longitude,datetime,rainfall,id,index_right,state
14147311,31.69,78.22,2021-06-07 21:00:00,0.000458,14147311,0,Himachal Pradesh
10277303,32.44,76.97,2018-06-04 21:00:00,0.000000,10277303,0,Himachal Pradesh
12188353,32.94,77.72,2019-08-04 02:00:00,0.018954,12188353,0,Himachal Pradesh
1674433,31.44,77.97,2011-07-08 15:00:00,0.007212,1674433,0,Himachal Pradesh
1795786,29.94,79.22,2011-07-20 05:00:00,0.103116,1795786,0,Uttarakhand
9955031,30.19,77.97,2017-09-04 04:00:00,0.023380,9955031,0,Uttarakhand
1308120,30.19,79.97,2011-06-03 17:00:00,0.000000,1308120,0,Uttarakhand
14715497,32.69,76.72,2021-08-01 01:00:00,0.001311,14715497,0,Himachal Pradesh
9342128,32.44,78.22,2017-07-07 17:00:00,0.000119,9342128,0,Himachal Pradesh
4796058,32.94,76.97,2013-08-31 06:00:00,0.007391,4796058,0,Himachal Pradesh


In [14]:
df_long['month'] = df_long['datetime'].dt.month
df_long['year']  = df_long['datetime'].dt.year
df_mon = df_long[df_long['month'].isin([6, 7, 8, 9])]

In [15]:
df_mon

,latitude,longitude,datetime,rainfall,id,index_right,state,month,year
192,30.44,77.47,2010-06-01 00:00:00,0.000000,192,0,Himachal Pradesh,6,2010
214,30.69,77.22,2010-06-01 00:00:00,0.000000,214,0,Himachal Pradesh,6,2010
215,30.69,77.47,2010-06-01 00:00:00,0.000000,215,0,Himachal Pradesh,6,2010
216,30.69,77.72,2010-06-01 00:00:00,0.000000,216,0,Himachal Pradesh,6,2010
236,30.94,76.97,2010-06-01 00:00:00,0.000000,236,0,Himachal Pradesh,6,2010
...,...,...,...,...,...,...,...,...,...
19192850,30.94,79.72,2024-09-30 23:00:00,0.000000,19192850,0,Uttarakhand,9,2024
19192867,31.19,78.22,2024-09-30 23:00:00,0.000229,19192867,0,Uttarakhand,9,2024
19192868,31.19,78.47,2024-09-30 23:00:00,0.001068,19192868,0,Uttarakhand,9,2024
19192870,31.19,78.97,2024-09-30 23:00:00,0.000953,19192870,0,Uttarakhand,9,2024


In [16]:
monthly_totals = (df_mon
    .groupby(['latitude','longitude','year','month'], as_index=False)
    ['rainfall'].sum()
    .rename(columns={'rainfall': 'monthly_rainfall'}))


In [17]:
monthly_totals

,latitude,longitude,year,month,monthly_rainfall
0,28.94,79.47,2010,6,36.459245
1,28.94,79.47,2010,7,743.314243
2,28.94,79.47,2010,8,525.090172
3,28.94,79.47,2010,9,565.702913
4,28.94,79.47,2011,6,149.030547
...,...,...,...,...,...
9955,32.94,77.72,2023,9,11.319392
9956,32.94,77.72,2024,6,7.781798
9957,32.94,77.72,2024,7,32.571660
9958,32.94,77.72,2024,8,116.347322


In [18]:
avg_monthly = (monthly_totals
    .groupby(['latitude','longitude','month'], as_index=False)
    ['monthly_rainfall'].mean()
    .rename(columns={'monthly_rainfall': 'avg_rainfall'}))


In [19]:
avg_monthly

,latitude,longitude,month,avg_rainfall
0,28.94,79.47,6,123.602878
1,28.94,79.47,7,422.281570
2,28.94,79.47,8,307.421369
3,28.94,79.47,9,168.159346
4,28.94,79.72,6,136.331751
...,...,...,...,...
659,32.94,76.97,9,65.814523
660,32.94,77.72,6,23.588139
661,32.94,77.72,7,59.983166
662,32.94,77.72,8,66.642982


## Global Moran's I Index


In [20]:
import numpy as np
import geopandas as gpd
from libpysal.weights import KNN
from esda.moran import Moran

# Create GeoDataFrame for coordinate-based weights
gdf = gpd.GeoDataFrame(avg_monthly, geometry=gpd.points_from_xy(avg_monthly.longitude, avg_monthly.latitude))
w = KNN.from_dataframe(gdf, k=8)  # 8 nearest neighbors (adjust k as needed)
w.transform = 'B'  # binary weights


In [ ]:
import folium
from libpysal.weights import KNN
from esda.moran import Moran

moran_results = {}
maps = {}

for m in [6, 7, 8, 9]:
    sub = gdf[gdf['month'] == m].copy()
    
    # rebuild weights
    w_sub = KNN.from_dataframe(sub, k=8)
    w_sub.transform = 'B'
    
    vals = sub['avg_rainfall'].values
    mi = Moran(vals, w_sub)
    
    moran_results[m] = {'I': mi.I, 'p-value': mi.p_norm}
    
    # make folium map
    fmap = folium.Map(location=[sub['latitude'].mean(), sub['longitude'].mean()], zoom_start=7)
    for _, row in sub.iterrows():
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=5,
            popup=f"Rainfall: {row['avg_rainfall']:.2f}",
            color='blue',
            fill=True,
            fill_opacity=0.6
        ).add_to(fmap)
    
    maps[m] = fmap


In [ ]:
maps[6].save('html_plots/june_rainfall_map.html')
maps[7].save('html_plots/july_rainfall_map.html')
maps[8].save('html_plots/august_rainfall_map.html') 
maps[9].save('html_plots/september_rainfall_map.html')

## Getis Ord Gi 

In [25]:
# Uttarakhand approximate bounds
uk = monthly_totals[
    (monthly_totals.latitude >= 28.43) & (monthly_totals.latitude <= 31.45) &
    (monthly_totals.longitude >= 77.34) & (monthly_totals.longitude <= 81.03)
]
# Himachal Pradesh approximate bounds
hp = monthly_totals[
    (monthly_totals.latitude >= 30.22) & (monthly_totals.latitude <= 33.25) &
    (monthly_totals.longitude >= 75.47) & (monthly_totals.longitude <= 79.04)
]


In [26]:
uk_sum = uk.groupby(['latitude','longitude'], as_index=False)['monthly_rainfall'].mean()
hp_sum = hp.groupby(['latitude','longitude'], as_index=False)['monthly_rainfall'].mean()


In [27]:
uk_gdf = gpd.GeoDataFrame(uk_sum, geometry=gpd.points_from_xy(uk_sum.longitude, uk_sum.latitude))
hp_gdf = gpd.GeoDataFrame(hp_sum, geometry=gpd.points_from_xy(hp_sum.longitude, hp_sum.latitude))
uk_w = KNN.from_dataframe(uk_gdf, k=8); uk_w.transform='B'
hp_w = KNN.from_dataframe(hp_gdf, k=8); hp_w.transform='B'


In [28]:
from esda.getisord import G_Local

# Uttarakhand Gi*
uk_G = G_Local(uk_gdf['monthly_rainfall'].values, uk_w, star=True)
uk_gdf['Z_score'] = uk_G.Zs
uk_gdf['p_sim'] = uk_G.p_sim
uk_gdf['hotspot'] = np.where((uk_gdf['p_sim']<0.05) & (uk_gdf['Z_score']>0), 'High (hotspot)',
                      np.where((uk_gdf['p_sim']<0.05) & (uk_gdf['Z_score']<0), 'Low (coldspot)', 
                               'Not Significant'))

# Himachal Pradesh Gi*
hp_G = G_Local(hp_gdf['monthly_rainfall'].values, hp_w, star=True)
hp_gdf['Z_score'] = hp_G.Zs
hp_gdf['p_sim'] = hp_G.p_sim
hp_gdf['hotspot'] = np.where((hp_gdf['p_sim']<0.05) & (hp_gdf['Z_score']>0), 'High (hotspot)',
                      np.where((hp_gdf['p_sim']<0.05) & (hp_gdf['Z_score']<0), 'Low (coldspot)', 
                               'Not Significant'))


In [ ]:
# Uttarakhand hotspot map
m_uk = folium.Map(location=[uk_gdf.latitude.mean(), uk_gdf.longitude.mean()], zoom_start=7)
for _, row in uk_gdf.iterrows():
    color = 'red' if row.hotspot=='High (hotspot)' else ('blue' if row.hotspot=='Low (coldspot)' else 'gray')
    folium.CircleMarker(
        (row.latitude, row.longitude), radius=1, color=color, fill=True, fill_opacity=0.6,
        popup=f"Rainfall: {row.monthly_rainfall:.1f}, Gi*: Z={row.Z_score:.2f}"
    ).add_to(m_uk)
m_uk.save('html_plots/Uttarakhand_hotspots.html')

# Himachal hotspot map (similar)
m_hp = folium.Map(location=[hp_gdf.latitude.mean(), hp_gdf.longitude.mean()], zoom_start=7)
for _, row in hp_gdf.iterrows():
    color = 'red' if row.hotspot=='High (hotspot)' else ('blue' if row.hotspot=='Low (coldspot)' else 'gray')
    folium.CircleMarker(
        (row.latitude, row.longitude), radius=1, color=color, fill=True, fill_opacity=0.6,
        popup=f"Rainfall: {row.monthly_rainfall:.1f}, Gi*: Z={row.Z_score:.2f}"
    ).add_to(m_hp)
m_hp.save('html_plots/Himachal_hotspots.html')
